# 20151230-predict-household-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-household-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [3]:
# Import standard packages.
import collections
import os
import pdb # TEST: Comment out pdb after testing.
import sys
# Import installed packages.
import matplotlib.pyplot as plt
import pandas as pd
# Import local packages.
sys.path.insert(
    0,
    os.path.join(os.path.curdir, 'stharrold.github.io/content/static/dsdemos'))
# TEST: Comment out autoreload after testing.
%reload_ext autoreload
%autoreload 2
import dsdemos as dsd
# IPython magic.
%matplotlib inline

## Globals

In [4]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-household-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_csv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13hdc.csv') # 'hdc' = 'housing DC'
path_ddict = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMSDataDict13.txt')

## Extract-transform-load

**TODO:**
* Just use pandas. Acknowledge dask.

In [5]:
%%time
with open(path_csv) as fobj:
    nlines = sum(1 for _ in fobj)
print("{path}:".format(path=path_csv))
print("size (MB) = {size:.1f}".format(size=os.path.getsize(path_csv)/1e6))
print("num lines = {nlines}".format(nlines=nlines))
df = pd.read_csv(path_csv)
print("df RAM usage (MB) = {mem:.1f}".format(mem=df.memory_usage().sum()/1e6))

/mnt/disk-20151227t211000z/www2-census-gov/programs-surveys/acs/data/pums/2013/5-Year/ss13hdc.csv:
size (MB) = 13.5
num lines = 17501
df RAM usage (MB) = 28.7
CPU times: user 412 ms, sys: 56 ms, total: 468 ms
Wall time: 466 ms


In [6]:
percentiles = [0.1587, 0.5000, 0.8413] # +1 std. dev., mean/median, -1 std. dev. for normal dist.
df.describe(percentiles=percentiles, include='all')

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
count,6561.000000,17500,1.750000e+04,17500,17500.000000,17500.000000,17500,17500,17500.000000,17500.000000,...,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000
unique,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,17500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,999.282731,NaN,2.011068e+12,5,56.427371,37.764171,3,11,1039364.231657,1048478.770229,...,17.050857,17.043486,17.050000,17.049029,17.048000,17.051543,17.053200,17.047029,17.046971,17.051486
std,1085.174484,NaN,1.401911e+09,0,55.291036,55.358495,0,0,31877.254257,29598.269890,...,17.593886,17.740566,17.534604,17.555515,17.558942,17.574232,17.623017,17.802284,17.267472,17.710924
min,0.000000,NaN,2.009000e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,200.000000,NaN,2.009001e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,790.000000,NaN,2.011001e+12,5,101.000000,-9.000000,3,11,1035725.000000,1054614.000000,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
84.1%,1500.000000,NaN,2.013000e+12,5,104.000000,104.000000,3,11,1086032.000000,1085467.000000,...,30.000000,30.000000,30.000000,31.000000,30.000000,30.000000,30.000000,30.000000,31.000000,30.000000


In [8]:
test = dsd.census.parse_pumsdatadict13(path=path_ddict)

In [11]:
test['notes']

['           -9999..09999 .Integer weight of housing unit',
 '           bb .N/A (not a worker or worker whose means of',
 '              .transportation to work was not car, truck,',
 '              .or van)',
 '           01 .Drove alone',
 '           02 .In 2-person carpool',
 '           03 .In 3-person carpool',
 '           04 .In 4-person carpool',
 '           05 .In 5-person carpool',
 '           06 .In 6-person carpool',
 '           07 .In 7-person carpool',
 '           08 .In 8-person carpool',
 '           09 .In 9-person carpool',
 '           10 .In 10-person or more carpool (Top-coded)',
 'NWAB       1 (UNEDITED - See "Employment Status Recode" (ESR))',
 '    Temporary absence from work',
 '           b .N/A (less than 16 years old/at work/on layoff)',
 '           1 .Yes',
 '           2 .No',
 '           3 .Did not report',
 'NWAV       1 (UNEDITED - See "Employment Status Recode" (ESR))',
 '    Available for work',
 '           b .N/A (less than 16 years/at work/

In [12]:
# TODO: Move to package.
# TODO: use example data dict as test.
# TODO: def ddict_to_json
# Entries in the data dictionary are "codes for variables", using the ACS terminology.
# https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.html
# Example data dictionary: http://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict13.txt
# The data dictionary is not all encoded in UTF-8. Replace encoding errors when found.
ddict = dict()
with open(path_ddict, encoding='utf-8', errors='replace') as fobj:
    # Data dictionary name is line 1.
    ddict['name'] = fobj.readline().strip()
    # Data dictionary date is line 2.
    ddict['date'] = fobj.readline().strip()    
    # Initialize flags to catch lines.
    catch_var_name = None
    catch_var_desc = None
    catch_var_code = None
    for line in fobj:
        line = line.rstrip()
        # Record type is section header 'HOUSING RECORD' or 'PERSON RECORD'.
        if line.strip() == 'HOUSING RECORD' or line.strip() == 'PERSON RECORD':
            record_type = line.strip()
            ddict[record_type] = dict()
        # A newline precedes a variable name.
        # A newline follows the last variable code.
        elif line == '':    
            catch_var_name = True
            catch_var_code = False
        # Variable name is 1 line with 0 space indent.
        # Variable name is followed by variable description.
        elif catch_var_name and not line.startswith(' ') and len(line.split()) == 2:
            (var_name, var_len) = line.strip().split()
            ddict[record_type][var_name] = dict()
            ddict[record_type][var_name]['length'] = var_len
            catch_var_name = False
            catch_var_desc = True
        # Variable description is 1 line with 4 space indent.
        # Variable description is followed by variable code(s).
        # Catch explicit instances of inconsistently formatted data.
        elif catch_var_desc:
            is_valid_desc = None
            if line.startswith(' '*4):
                is_valid_desc = True
            # Example inconsistent format case:
            # ADJHSG     7      
            # Adjustment factor for housing dollar amounts (6 implied decimal places)
            elif 'ADJ' in var_name and 'Adjustment factor' in line:
                is_valid_desc = True
            else:
                pass
            if is_valid_desc:
                var_desc = line.strip()
                ddict[record_type][var_name]['description'] = var_desc
                catch_var_desc = False
                catch_var_code = True
            if is_valid_desc is None:
                raise AssertionError("Program error. `is_valid_desc` must be set within if-elif-else.")
        # Variable code(s) is 1+ line with 8+ space indent. Example:
        # NP         2      
        #     Number of person records following this housing record
        #                00 .Vacant unit
        #                01 .One person record (one person in household or  
        #                   .any person in group quarters)
        #            02..20 .Number of person records (number of persons in
        #                   .household)
        # The last variable code is followed by a newline.
        # Catch explicit instances of inconsistently formatted data.
        elif catch_var_code:
            is_valid_code = None
            if line.startswith(' '*8):
                # Example case: ".any person in group quarters)"
                if line.strip().startswith('.'):
                    var_code_desc = line.strip().lstrip('.')
                    ddict[record_type][var_name][var_code] += ' '+var_code_desc
                    is_valid_code = False
                # Example inconsistent format case:
                # DEAR<tab>   1
                #     Hearing difficulty
                #            1. Yes
                #            2. No
                elif 'DEAR' in var_name and ('1. Yes' in line or '2. No' in line):
                    (var_code, var_code_desc) = line.strip().split(sep='. ', maxsplit=1)
                    is_valid_code = True
                # Example inconsistent format case:
                # MARHYP     4
                #     Year last married
                #            bbbb. N/A (age less than 15 years; never married)
                #            1932 .1932 or earlier (Bottom-coded)
                #            1933 .1933
                elif 'MARHYP' in var_name and 'bbbb. N/A' in line:
                    (var_code, var_code_desc) = line.strip().split(sep='. ', maxsplit=1)
                    is_valid_code = True
                # Example case: "01 .One person record (one person in household or"
                else:
                    (var_code, var_code_desc) = line.strip().split(sep=' .', maxsplit=1)
                    is_valid_code = True
            # Example inconsistent format case:
            # RWAT       1      
            #     Hot and cold running water
            #            b .N/A (GQ)
            #            1 .Yes
            #            2 .No
            # <tab>   9. Case is from Puerto Rico, RWAT not applicable
            elif 'RWAT' in var_name and '9. Case is from' in line:
                (var_code, var_code_desc) = line.strip().split(sep='. ', maxsplit=1)
                is_valid_code = True
            # Example inconsistent format:
            # SMP        5      
            #     Total payment on all second and junior mortgages and home equity loans 
            #     (monthly amount)
            #                   bbbbb .N/A (GQ/vacant/not owned or being bought/
            #                         ./no second or junior mortgages or home equity loans)
            #            00001..99999 .$1 to $99999 (Rounded and top-coded)
            elif 'SMP' in var_name and '(monthly amount)' in line:
                var_desc = line.strip()
                ddict[record_type][var_name]['description'] += ' '+var_desc
                is_valid_code = False
            else:
                is_valid_code = False
            if is_valid_code:
                ddict[record_type][var_name][var_code] = var_code_desc
            if is_valid_code is None:
                raise AssertionError("Program error. `is_valid_code` must be set within if-elif-else.")
        # Variable note is preceded by newline.
        # Variable note is 1 line.
        # Variable note is followed by newline.
        elif line.startswith('Note:'):
            var_note = line.lstrip('Note:').strip()
            ddict[record_type][var_name]['note'] = var_note
ddict

{'HOUSING RECORD': {'ACCESS': {'1': 'Yes, with subscription to an Internet service',
   '2': 'Yes, without a subscription to an Internet service',
   '3': 'No Internet access at this house, apartment, or mobile home',
   'b': 'N/A (GQ)',
   'description': 'Access to the Internet',
   'length': '1'},
  'ACR': {'1': 'House on less than one acre',
   '2': 'House on one to less than ten acres',
   '3': 'House on ten or more acres',
   'b': 'N/A (GQ/not a one-family house or mobile home)',
   'description': 'Lot size',
   'length': '1'},
  'ADJHSG': {'1000000': '2013 factor (1.000000)',
   'description': 'Adjustment factor for housing dollar amounts (6 implied decimal places)',
   'length': '7',
   'note': 'The value of ADJHSG inflation-adjusts reported housing costs to 2013 dollars and applies to variables CONP, ELEP, FULP, GASP, GRNTP, INSP, MHP, MRGP, SMOCP, RNTP, SMP, and WATP in the housing record. ADJHSG does not apply to AGS and TAXP because they are categorical variables that should

In [15]:
ddict['PERSON RECORD'].keys()

dict_keys(['FDEYEP', 'FDISP', 'RACWHT', 'DRAT', 'PWGTP', 'GCL', 'GCM', 'FHINS3P', 'SCHL', 'PWGTP47', 'PWGTP15', 'CIT', 'PWGTP26', 'PWGTP32', 'PWGTP79', 'FPUBCOVP', 'SCIENGRLP', 'FANCP', 'POVPIP', 'OCCP', 'PWGTP65', 'FMARP', 'RACAIAN', 'HINS2', 'PWGTP24', 'PWGTP52', 'AGEP', 'PWGTP6', 'FSCHGP', 'ANC1P', 'HISP', 'SSP', 'PRIVCOV', 'SCHG', 'PWGTP70', 'FESRP', 'PWGTP72', 'PWGTP22', 'PWGTP21', 'PWGTP51', 'PWGTP28', 'FHINS5C', 'CITWP', 'PINCP', 'DEAR', 'FHISP', 'FLANP', 'FMIGP', 'FSSP', 'MSP', 'PWGTP3', 'RAC1P', 'PWGTP19', 'FDDRSP', 'FOIP', 'FDEARP', 'SERIALNO', 'YOEP', 'DRATX', 'PWGTP43', 'DREM', 'INTP', 'JWAP', 'FPAP', 'ESP', 'PWGTP53', 'FWKHP', 'PWGTP59', 'PWGTP42', 'PAP', 'PERNP', 'FWKLP', 'PWGTP75', 'PWGTP33', 'RACSOR', 'PWGTP77', 'PWGTP35', 'FDOUTP', 'MARHW', 'SPORDER', 'PWGTP23', 'FHINS7P', 'LANP', 'FHINS4P', 'GCR', 'SOCP', 'FFERP', 'MLPB', 'COW', 'NOP', 'FGCMP', 'MLPH', 'JWMNP', 'ADJINC', 'MLPI', 'PWGTP27', 'POBP', 'RELP', 'FJWMNP', 'FDREMP', 'FINDP', 'FRETP', 'PWGTP71', 'WRK', 'FENGP'

## Export ipynb to html

In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)